# Employee Queries:

"Show me all employees in the Sales department."
"Who is the manager of the Engineering department?"
"List all employees hired after 2023-01-01."
"What is the total salary expense for the IT department?"
Other Variations:

"Employees in the HR department."
"Manager of the Finance department."
"Employees hired after 2022-06-15."
"Total salary expense for the Marketing department."

In [1]:
import os

# Check if the database file 'chat_assistant.db' exists in the current directory
print(os.path.exists("chat_assistant.db"))



True


In [3]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('chat_assistant.db')
cursor = conn.cursor()

# Retrieve and display all table names in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in database:", tables)

# Close the database connection
conn.close()



Tables in database: [('departments',), ('sqlite_sequence',), ('employees',)]


In [5]:
import sqlite3
import re
import nltk
import pandas as pd
from datetime import datetime

# Download punkt tokenizer for NLP
nltk.download('punkt')

# Database connection
def connect_db():
    conn = sqlite3.connect('chat_assistant.db')
    return conn

# Load CSV data into SQLite
def load_csv_to_db(csv_path):
    df = pd.read_csv(csv_path)
    conn = connect_db()
    df.to_sql('employees', conn, if_exists='replace', index=False)
    conn.close()

# Queries

def get_employees_in_department(department):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT name FROM employees 
        WHERE department_id = (SELECT id FROM departments WHERE name = ?)
    """, (department,))
    employees = cursor.fetchall()
    conn.close()
    
    if not employees:
        return "No employees found."
    
    return "\n".join([f"{i+1}. {employee[0]}" for i, employee in enumerate(employees)])


def get_manager_of_department(department):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT manager FROM departments WHERE name = ?", (department,))
    manager = cursor.fetchone()
    conn.close()
    return manager[0] if manager else "No manager found."


def get_employees_hired_after(date):
    try:
        hire_date = datetime.strptime(date, '%Y-%m-%d').date()
    except ValueError:
        return "Invalid date format. Use YYYY-MM-DD."
    
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM employees WHERE hire_date > ?", (hire_date,))
    employees = cursor.fetchall()
    conn.close()
    
    if not employees:
        return "No employees found."
    
    return "\n".join([f"{i+1}. {employee[0]}" for i, employee in enumerate(employees)])


def get_total_salary_expense(department):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT SUM(salary) FROM employees 
        WHERE department_id = (SELECT id FROM departments WHERE name = ?)
    """, (department,))
    total_salary = cursor.fetchone()
    conn.close()
    return total_salary[0] if total_salary and total_salary[0] else "No salary data found."

# Query Processor
def process_query(query):
    query = query.lower()

    if "employees in the" in query and "department" in query:
        match = re.search(r"employees in the (.*?) department", query)
        if match:
            return get_employees_in_department(match.group(1).capitalize())
    
    elif "manager of the" in query and "department" in query:
        match = re.search(r"manager of the (.*?) department", query)
        if match:
            return get_manager_of_department(match.group(1).capitalize())
    
    elif "employees hired after" in query:
        match = re.search(r"employees hired after (\d{4}-\d{2}-\d{2})", query)
        if match:
            return get_employees_hired_after(match.group(1))
    
    elif "total salary expense for the" in query:
        match = re.search(r"total salary expense for the (.*?) department", query)
        if match:
            return get_total_salary_expense(match.group(1).capitalize())
    
    return "Sorry, I didn't understand that query."

# Start Chat Assistant
def start_chat():
    print("Hello! I'm your chat assistant. How can I assist you?")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Goodbye! Have a nice day!")
            break
        response = process_query(user_input)
        print("Assistant:\n", response)

if __name__ == "__main__":
    #load_csv_to_db('employees.csv')  # Load data
    load_csv_to_db(r"C:\Users\krish\Downloads\employees.csv") 
    start_chat()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Hello! I'm your chat assistant. How can I assist you?


You:  Show me all employees in the Sales department.


Assistant:
 1. Teresa Barber
2. Michael Andersen
3. Stephanie Knight
4. Cassidy Spears
5. Jennifer Horn
6. Brian Webster
7. Sharon Cook
8. Megan Rodriguez
9. Suzanne Parsons
10. Arthur White
11. Julia Smith
12. Dr. William Woodard
13. Andrea Mitchell
14. Valerie Maddox
15. Jillian Bryant
16. Jennifer Andrews
17. Pedro Munoz
18. Joel Osborne
19. David Buchanan MD
20. Jacob Espinoza
21. Christine Stevens
22. David Patterson
23. Karen May
24. Monica Rivera
25. Parker Evans
26. Jonathan Buck
27. Jeremy Owens
28. Alexander Johnson
29. Karen Cruz
30. Kimberly Soto
31. Adrian Williams
32. Tasha Greene
33. Daniel Hurst
34. Jacob Harris
35. Debra Conley
36. Michael Stewart
37. Diana Smith
38. Roy Carroll
39. Lisa Barker
40. Mary Bryant
41. Melissa Kelly
42. Denise Rosales
43. Robert Turner
44. Ashley Lutz
45. Daniel Vasquez
46. Alexandra Gonzalez
47. Valerie Bates
48. Courtney Baker
49. Anthony Curry MD
50. Shawn Lee
51. Lisa Wagner
52. David Taylor
53. Beverly Lewis
54. Tammy Flowers
55. Chris

You:  Who is the manager of the Engineering department?


...

Assistant:
 Sorry, I didn't understand that query.


You:  Total salary expense for the Marketing department.


Assistant:
 4479625


You:  quit


Goodbye! Have a nice day!
